# Setup

In [25]:
from numpy import random
import itertools
import numpy
from collections import namedtuple, Mapping
import six

# Graph

## Generate graph

In [2]:
NUM_SPINS = 26
NUM_STATES = 10
OMP_THREADS = 4

In [16]:
graph_rand = random.rand(NUM_SPINS, NUM_SPINS) * 10 - 5 # pylint: disable=no-member

In [17]:
graph_rand

array([[-2.96810066e-01, -2.83242981e+00,  3.75228427e+00,
         3.16923904e+00, -3.68657155e+00,  4.22657061e+00,
        -3.41202827e+00,  3.14470396e+00,  5.73721480e-01,
        -3.06185443e+00, -2.26294988e+00,  4.23960836e+00,
        -3.04965522e+00, -3.46398023e+00, -1.23067412e+00,
        -3.75010323e+00, -1.59949782e+00,  1.19106312e+00,
         4.20778890e+00,  1.99356169e+00, -9.15417179e-02,
         4.30515524e+00,  3.58472177e+00,  2.60578839e-01,
        -9.11638158e-01, -4.12118822e+00],
       [-1.13229439e-01, -2.33928175e+00,  1.15201617e+00,
         3.60367603e+00,  4.03424770e+00,  4.31974461e+00,
        -3.95103262e+00, -2.27809062e+00,  2.09316736e+00,
         3.88849787e+00, -3.54028720e+00,  2.77592406e+00,
        -1.99639188e+00,  2.03317146e+00, -2.16411115e+00,
        -3.94002961e+00,  3.58086745e+00, -1.84091652e+00,
        -2.03630095e+00,  3.25967819e+00,  4.86451083e-01,
        -2.31804708e-01, -2.58023822e+00, -1.17208303e-01,
         8.99

In [18]:
graph = {}
for i in range(NUM_SPINS):
    graph[(i, i)] = random.rand() * 10 - 5
    for j in range(i, NUM_SPINS):
        graph[(i, j)] = random.rand() * 10 - 5

In [19]:
graph

{(0, 0): 1.0406526183980365,
 (0, 1): 3.1947258121558324,
 (0, 2): 2.209124115173241,
 (0, 3): 1.256206606929985,
 (0, 4): -3.7628891065271706,
 (0, 5): -2.4807413474274718,
 (0, 6): 0.15699879886464707,
 (0, 7): 4.138051367633825,
 (0, 8): 0.5702848218247016,
 (0, 9): -2.918340093799018,
 (0, 10): -4.752257830826544,
 (0, 11): 4.026118157090993,
 (0, 12): -4.136769818212684,
 (0, 13): 2.053584295969908,
 (0, 14): 2.247156380387551,
 (0, 15): 3.5643104197492708,
 (0, 16): 2.9269827230823697,
 (0, 17): -3.045086621889743,
 (0, 18): 2.0557478339766018,
 (0, 19): -0.38055377322405715,
 (0, 20): -1.7797636293940977,
 (0, 21): 2.5533334560463103,
 (0, 22): -4.673753061253242,
 (0, 23): 4.472987801891019,
 (0, 24): 0.4723139470406199,
 (0, 25): 4.683211923822078,
 (1, 1): -4.287793088023269,
 (1, 2): 0.873318347670077,
 (1, 3): 3.9481469610229443,
 (1, 4): 2.2043531252372626,
 (1, 5): 2.78880378221615,
 (1, 6): 1.4982449317801354,
 (1, 7): 0.6178093477051059,
 (1, 8): -3.2306842436860572,
 (

In [11]:
# search
omp_threads = 4
num_states=10
chunk_size=20
energies_only=False

In [27]:
read_graph(graph)

IsingProblem(matrix=array([[-4.11196838e+00, -4.80449140e+00, -1.39906436e+00,
         2.30959164e+00,  1.88305291e+00, -3.90501837e+00,
        -1.54875567e+00,  5.39004680e-01, -3.91874047e+00,
         6.91324367e-01,  2.13005629e-01, -3.43376656e+00,
        -2.70648957e+00, -1.77933690e+00, -9.14665351e-01,
        -9.35474098e-02, -5.94875814e-01, -4.63485870e+00,
         4.79864588e+00,  9.03512758e-01,  4.75426414e-01,
        -4.49661948e+00, -2.89028794e-01,  3.10215367e+00,
        -2.47115234e+00,  3.12436422e+00],
       [ 0.00000000e+00, -1.51264197e+00,  3.70992684e+00,
         2.86916541e+00,  3.62379589e+00,  9.71822155e-01,
         4.34001546e+00,  4.11436802e+00, -4.71810277e+00,
        -4.44575896e+00,  3.21281205e+00, -3.30431197e+00,
        -4.11601826e+00,  1.50067150e+00,  2.90358480e-02,
         1.60539333e+00, -8.80953538e-01,  4.43930428e+00,
        -7.26618413e-01, -1.58313264e+00,  4.71478001e+00,
         3.60825048e+00, -4.67940995e+00,  1.2939470

In [21]:


IsingProblem = namedtuple('IsingProblem', ['matrix', 'spin_labels'])

def read_graph(graph):
    """Read Hamiltonian and spin labels from given graph.

    :param graph: a definition of Ising problem in one of the following formats:
     - as a Mapping of spin indices (i, j) to the corresponding coupler (or
       spin weight if i == j).
     - as a row-format, i.e. an iterable of (i, j, value) with analogous meaning
       as in Mapping format.
     - as a square matrix (numpy.ndarray or nested lists) such that graph[i, j] corresponds
       to coupling between spins i and j (or a spin weight if i == j).
    :returns: a named tuplew with matrix and spin_labels components. The matrix
     is a square upper triangular 2D numpy.ndarray, upper triangular, encoding
     system Hamiltonian (without lower triangle unneeded for computations).
     The spin_labels is a tuple of integers mapping spins of system represented
     by the matrix into spins into original graph.
    :rtype: IsingProblem
    """

    if isinstance(graph, Mapping):
        return _ising_from_mapping(graph)
    detected_shape = numpy.ma.shape(graph)
    if len(detected_shape) != 2:
        raise ValueError('Unsupported graph format specified. Consult docs.')
    if detected_shape[1] == 3 and has_possibly_spin_labels(graph):
        return _ising_from_rows(graph)

    if detected_shape[0] == detected_shape[1]:
        return _ising_from_matrix(graph)

    raise ValueError('Unsupported graph format specified. Consult docs.')

def _ising_from_matrix(graph):
    graph = numpy.array(graph)
    jh_matrix = numpy.zeros(graph.shape)
    for i in range(graph.shape[0]):
        for j in range(graph.shape[1]):
            first, second = sorted((i, j))
            jh_matrix[first, second] += graph[i, j]
    return IsingProblem(jh_matrix, tuple(range(jh_matrix.shape[0])))

def _ising_from_mapping(graph_map):
    seen_spins = set()
    for i, j in graph_map:
        seen_spins.add(i)
        seen_spins.add(j)
    system_size = len(seen_spins)
    jh_matrix = numpy.zeros(shape=(system_size, system_size))
    spin_labels = tuple(sorted(seen_spins))
    spin_map = {spin: idx for idx, spin in enumerate(spin_labels)}
    for (i, j), value in six.iteritems(graph_map):
        first, second = sorted((spin_map[i], spin_map[j]))
        jh_matrix[first, second] += value
    return IsingProblem(jh_matrix, spin_labels)

def ising_to_qubo(ham):
    qubo = numpy.zeros(ham.shape)
    constant = 0
    for i in range(qubo.shape[0]):
        qubo[i, i] = 2 * ham[i, i]
        constant +=  ham[i, i]
        for j in range(qubo.shape[0]):
            if i != j:
                low, high = sorted((i, j))
                constant -= ham[low, high] * 0.5
                qubo[i, i] -= 2 * ham[low, high]
                qubo[low, high] += ham[low, high] * 2
    return qubo, constant

def max_chunk_size_for_method(method):
    """Determine max chunk size given computation method.

    :param method: method that shall be used for computations, either 'cpu' or 'gpu' (case
     insensitive).
    :type method: str
    :returns: maximum chunk size for given computation method
    :rtype: int
    :raises ValueError: if unrecognized computation method is passed.
    """
    method = method.lower()
    if method == 'cpu':
        mem_bytes = psutil.virtual_memory().available
    elif method == 'gpu':
        mem_bytes = gpusearch.get_device_properties()
    else:
        raise ValueError('Unknown computation method: %s', method)
    return max_chunk_size(mem_bytes)

In [26]:
num_states=10
method='GPU', # 'CPU'
show_progress=True
chunk_size=20

jh_mat, labels = read_graph(graph)

qubo, const = ising_to_qubo(jh_mat)

# method = choose_method(gpusearch, **kwargs)

if chunk_size is None:
    chunk_size = max_chunk_size_for_method(method)

if chunk_size > jh_mat.shape[0]:
    chunk_size = jh_mat.shape[0]

if num_states > 2 * 2 ** chunk_size:
    num_states = 2 ** chunk_size

In [28]:
jh_mat

array([[ 1.04065262,  3.19472581,  2.20912412,  1.25620661, -3.76288911,
        -2.48074135,  0.1569988 ,  4.13805137,  0.57028482, -2.91834009,
        -4.75225783,  4.02611816, -4.13676982,  2.0535843 ,  2.24715638,
         3.56431042,  2.92698272, -3.04508662,  2.05574783, -0.38055377,
        -1.77976363,  2.55333346, -4.67375306,  4.4729878 ,  0.47231395,
         4.68321192],
       [ 0.        , -4.28779309,  0.87331835,  3.94814696,  2.20435313,
         2.78880378,  1.49824493,  0.61780935, -3.23068424,  2.26511907,
         1.68338871, -1.34246099,  4.0554458 , -1.92325431,  1.54044955,
        -2.97047273, -4.37258747, -2.32819348, -3.07589437, -2.01212306,
        -3.44083895,  0.18338674,  1.9242701 ,  2.82573284,  4.03777884,
         2.67379416],
       [ 0.        ,  0.        , -1.8201839 , -0.66916865, -4.2288256 ,
        -2.49056097, -2.02450173, -3.9420841 ,  1.92479159, -2.99797538,
         1.7596786 ,  1.19361969, -4.49837946, -0.88552282, -2.24302818,
       

In [73]:
qubo

array([[ 2.96767213e+01, -1.92179656e+01, -5.59625743e+00,
         9.23836655e+00,  7.53221165e+00, -1.56200735e+01,
        -6.19502270e+00,  2.15601872e+00, -1.56749619e+01,
         2.76529747e+00,  8.52022516e-01, -1.37350662e+01,
        -1.08259583e+01, -7.11734760e+00, -3.65866140e+00,
        -3.74189639e-01, -2.37950326e+00, -1.85394348e+01,
         1.91945835e+01,  3.61405103e+00,  1.90170566e+00,
        -1.79864779e+01, -1.15611518e+00,  1.24086147e+01,
        -9.88460937e+00,  1.24974569e+01],
       [ 0.00000000e+00, -2.33732275e+01,  1.48397073e+01,
         1.14766616e+01,  1.44951836e+01,  3.88728862e+00,
         1.73600618e+01,  1.64574721e+01, -1.88724111e+01,
        -1.77830358e+01,  1.28512482e+01, -1.32172479e+01,
        -1.64640730e+01,  6.00268600e+00,  1.16143392e-01,
         6.42157333e+00, -3.52381415e+00,  1.77572171e+01,
        -2.90647365e+00, -6.33253055e+00,  1.88591201e+01,
         1.44330019e+01, -1.87176398e+01,  5.17578836e+00,
         2.82

In [30]:
def spin_variable_generator(n):
    for s in itertools.product([-1,1],repeat=n):
        yield s
        
def qubo_variable_generator(n):
    for s in itertools.product([0,1],repeat=n):
        yield s

In [70]:
for s in qubo_variable_generator(3):
    print(s)

(0, 0, 0)
(0, 0, 1)
(0, 1, 0)
(0, 1, 1)
(1, 0, 0)
(1, 0, 1)
(1, 1, 0)
(1, 1, 1)


In [ ]:
if energies_only:
    energies = cpusearch.find_lowest_energies_only(qubo,
                                                   chunk_size,
                                                   num_states,
                                                   callback=callback)
else:
    energies, state_reprs = cpusearch.find_lowest(qubo,
                                                  chunk_size,
                                                  num_states,
                                                  callback=callback)

In [38]:
len(jh_mat), len(jh_mat[0])

(26, 26)

In [44]:
num_states

10

In [28]:
def search(graph, num_states=10, energies_only=False, **kwargs):
    """Find lowest energies and corresponding states of Ising Model specified by Jh.

    :param Jh: matrix encoding hamiltonian of the sytem. Should be real and symmetric.
    :type Jh: numpy.ndarray or array-like
    :param sweep_size: number of bits being changed during a single sweep during exhaustive
     search. Has to obey `1 <= sweep_size <= Jh.shape[0]`.
    :type sweep_size: int
    :param num_states: how many lowest states to keep. Default is 10.
    :type num_states:
    :returns: namedtuple with components `states` and `energies` holding computed states of
     lowest energy and corresponding energies.
    :rtype: :py:class:`EnergiesAndStates
    """
    logger = logging.getLogger('ising')

    jh_mat, labels = read_graph(graph)

    qubo, const = ising_to_qubo(jh_mat)
    method = choose_method(gpusearch, **kwargs)
    logger.info('Choosen computation mehtod: %s', method)

    if 'chunk_size' in kwargs:
        chunk_size = kwargs['chunk_size']
        logger.debug('Chunk size given explicitly as %f.', chunk_size)
    else:
        logger.debug('Deducing chunk size from the available memory.')
        chunk_size = max_chunk_size_for_method(method)

    if chunk_size > jh_mat.shape[0]:
        logger.debug('Clipping chunk size to the number of spins.')
        chunk_size = jh_mat.shape[0]

    logger.info('Chunk size exponent that will be used: %d', chunk_size)

    if num_states > 2 * 2 ** chunk_size:
        logger.warning('Requested more states than two chunks. Clipping at 2 ** (chunk size)')
        num_states = 2 ** chunk_size

    if kwargs.get('show_progress', False):
        pbar = progressbar.ProgressBar(max_value=2 ** (jh_mat.shape[0]-chunk_size))
        callback = pbar.update
    else:
        pbar = None
        callback = dummy_callback

    if method.lower() == 'cpu':
        logger.info('Running CPU search: Jh.shape=%s, chunk_size=%d, num_states=%d',
                    jh_mat.shape, chunk_size, num_states)
        callback(0)
        if energies_only:
            energies = cpusearch.find_lowest_energies_only(qubo,
                                                           chunk_size,
                                                           num_states,
                                                           callback=callback)
        else:
            energies, state_reprs = cpusearch.find_lowest(qubo,
                                                          chunk_size,
                                                          num_states,
                                                          callback=callback)
    else:
        logger.info('Running GPU search: Jh.shape=%s, num_states=%d, chunk_size=%d',
                    jh_mat.shape, num_states, chunk_size)
        callback(0)
        if energies_only:
            energies = gpusearch.find_lowest_energies_only(qubo,
                                                           chunk_size,
                                                           num_states,
                                                           callback=callback)
        else:
            energies, state_reprs = gpusearch.find_lowest(qubo,
                                                          chunk_size,
                                                          num_states,
                                                          callback=callback)
    if pbar is not None:
        pbar.finish()
    if energies_only:
        state_reprs = None
    return EnergiesAndStates(raw_states=state_reprs, labels=labels, energies=energies + const)

In [33]:
for k in qubo_variable_generator(3):
    print(k)

(0, 0, 0)
(0, 0, 1)
(0, 1, 0)
(0, 1, 1)
(1, 0, 0)
(1, 0, 1)
(1, 1, 0)
(1, 1, 1)


In [49]:
%%timeit

NUM_SPINS = 20

graph = {}
for i in range(NUM_SPINS):
    graph[(i, i)] = random.rand() * 10 - 5
    for j in range(i, NUM_SPINS):
        graph[(i, j)] = random.rand() * 10 - 5

jh_mat, labels = read_graph(graph)

qubo, const = ising_to_qubo(jh_mat)

res = []

for k, q in enumerate(qubo_variable_generator(NUM_SPINS)):
    F = 0
    for i in range(qubo.shape[0]):
        F -= qubo[i,i]*q[i]
        for j in range(qubo.shape[0]):
            low, high = sorted([i, j])
            F -= qubo[low, high] * q[i] * q[j]
    res.append(F)
#     if k == NUM_SPINS:
#         break
len(res), res

3min 58s ± 588 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
    qubo = numpy.zeros(ham.shape)
    constant = 0
    for i in range(qubo.shape[0]):
        qubo[i, i] = 2 * ham[i, i]
        constant +=  ham[i, i]
        for j in range(qubo.shape[0]):
            if i != j:
                low, high = sorted((i, j))
                constant -= ham[low, high] * 0.5
                qubo[i, i] -= 2 * ham[low, high]
                qubo[low, high] += ham[low, high] * 2
    return qubo, constant


In [78]:
qubo.shape[0]

26

In [79]:
qubo

array([[ 2.96767213e+01, -1.92179656e+01, -5.59625743e+00,
         9.23836655e+00,  7.53221165e+00, -1.56200735e+01,
        -6.19502270e+00,  2.15601872e+00, -1.56749619e+01,
         2.76529747e+00,  8.52022516e-01, -1.37350662e+01,
        -1.08259583e+01, -7.11734760e+00, -3.65866140e+00,
        -3.74189639e-01, -2.37950326e+00, -1.85394348e+01,
         1.91945835e+01,  3.61405103e+00,  1.90170566e+00,
        -1.79864779e+01, -1.15611518e+00,  1.24086147e+01,
        -9.88460937e+00,  1.24974569e+01],
       [ 0.00000000e+00, -2.33732275e+01,  1.48397073e+01,
         1.14766616e+01,  1.44951836e+01,  3.88728862e+00,
         1.73600618e+01,  1.64574721e+01, -1.88724111e+01,
        -1.77830358e+01,  1.28512482e+01, -1.32172479e+01,
        -1.64640730e+01,  6.00268600e+00,  1.16143392e-01,
         6.42157333e+00, -3.52381415e+00,  1.77572171e+01,
        -2.90647365e+00, -6.33253055e+00,  1.88591201e+01,
         1.44330019e+01, -1.87176398e+01,  5.17578836e+00,
         2.82